In [88]:
import importlib
import pandas as pd
importlib.reload(cs_baseline)

<module 'cs_baseline' from '/home/chez/projects/cs_dash/cs_baseline.py'>

In [6]:
  cs_baseline.cs_baseline(2022,[1,2,3],3
,"Par A",[165,265,365], 0 , 0 , 0 , 0, 0,50000,0,0
,"Par B",[0  ,0  ,0  ], 0 , 0 , 0 , 0, 0,150000,0,0 )

combined_csi 145874.0
a_income_pct 0.1572
a_cost_pct 0.45
a_cost_pct 0.76
a_cost_pct 1
basic_coc 37445.4
a_form_liab 0.0
b_form_liab 21698.36112
a_total_liability -21698.36112


-21698.36112

In [101]:
import cProfile
#600000 repititions 45 seconds
cProfile.run('for i in range(600000):\
             cs_baseline(2022,[1,2,3],3\
,"Par A",[165,265,365], 0 , 0 , 0 , 0, 0,50000,0,0\
,"Par B",[0  ,0  ,0  ], 0 , 0 , 0 , 0, 0,150000,0,0 )')

In [94]:
cs_validated = pd.read_csv("cs_validated.csv")
(cs_validated.columns)

Index(['Case_id', 'liability', 'payer', 'Assess_year', 'A_ATI', 'B_ATI',
       'A_ISP', 'B_ISP', 'Age1', 'Age2', 'Age3', 'Age4', 'Age5', 'A_CN1',
       'A_CN2', 'A_CN3', 'A_CN4', 'A_CN5', 'A_RelDep_12l', 'A_RelDep_13p',
       'B_RelDep_12l', 'B_RelDep_13p', 'A_othercase_n', 'A_othercase_12l',
       'A_othercase_13p', 'B_othercase_n', 'B_othercase_12l',
       'B_othercase_13p', 'a_oth_lsc', 'b_oth_lsc', 'a_farnta', 'b_farnta'],
      dtype='object')

In [95]:
cs_validated["ages"] = cs_validated[['Age1', 'Age2', 'Age3', 'Age4', 'Age5']].values.tolist()
cs_validated["nchild"] = 5 - cs_validated[['Age1', 'Age2', 'Age3', 'Age4', 'Age5']].isnull().sum(axis=1)
cs_validated["a_name"] = "A"
cs_validated["b_name"] = "B"
cs_validated["a_cn"] = cs_validated[['A_CN1','A_CN2', 'A_CN3', 'A_CN4', 'A_CN5']].values.tolist()
cs_validated["b_cn"] = [[0,0,0,0,0]]*len(cs_validated)
cs_validated["a_ati"] = cs_validated['A_ATI'].replace('[\$,]', '', regex=True).astype(float)
cs_validated["b_ati"] = cs_validated['B_ATI'].replace('[\$,]', '', regex=True).astype(float)
cs_validated["a_isp"] = cs_validated['A_ISP'].replace('Yes', '1').replace('No', '0').astype(int)
cs_validated["b_isp"] = cs_validated['B_ISP'].replace('Yes', '1').replace('No', '0').astype(int)
cs_validated["csv_liability"] = cs_validated['liability'].replace('[\$,]', '', regex=True).replace('[-]', '0', regex=True).astype(float)

In [31]:
#cs_validated["a_cn"]

In [96]:
cs_validated = cs_validated.rename(columns={'Assess_year': 'year', 'A_othercase_n': 'a_othercase_n', 
'A_RelDep_12l':'a_reldep_12l', 'A_RelDep_13p': 'a_reldep_13p', 
'A_othercase_12l': 'a_othercase_12l', 'A_othercase_13p': 'a_othercase_13p', 'B_othercase_n': 'b_othercase_n', 
'B_RelDep_12l': 'b_reldep_12l','B_RelDep_13p': 'b_reldep_13p',  
'B_othercase_12l': 'b_othercase_12l', 'B_othercase_13p': 'b_othercase_13p'})

In [117]:
importlib.reload(cs_baseline)
wrong = 0
for row in cs_validated.itertuples():
    results = cs_baseline.cs_baseline(**row._asdict() )
    csv_liability = row._asdict()["csv_liability"]
    baseline_liability =results["liability"]
    if (csv_liability - abs(baseline_liability)) > 5 :
        if wrong > 2:
            print(baseline_liability,results)
            print(row)
            break
        wrong = wrong +1 
print(wrong)
    #a_selected_liability


-1569.0 {'a_ati_lessssa': 0, 'b_ati_lessssa': 24141.0, 'a_ati_lessreldep': 0, 'b_ati_lessreldep': 18347.16, 'a_csi': 0, 'b_csi': 18347.16, 'combined_csi': 18347.0, 'a_income_pct': 0.0, 'b_income_pct': 1.0, 'a_mc_cost_pc_12l': 0.0, 'a_mc_cost_pc_13p': 0.0, 'b_mc_cost_pc_12l': 1238.4333000000001, 'b_mc_cost_pc_13p': 1467.7728, 'basic_coc': 5412.365000000001, 'a_init_liab': [0.0, 0.0, 0.0, 0.0], 'b_init_liab': [0.0, 383.9109750000001, 1467.76, 1238.4225000000001], 'a_mc_cap': [0.0, 0.0, 0.0, 0.0], 'b_mc_cap': [0.0, 383.91432300000014, 1467.7728, 1238.4333000000001], 'a_isp': 0, 'a_ati': 17833.0, 'b_isp': 0, 'b_ati': 51204.0, 'a_unit_far': 1521.0, 'b_unit_far': 1521.0, 'a_cost_pct': [0, 0.31, 1, 1], 'b_cost_pct': [1, 0.69, 0, 0], 'a_form_far': [1521.0, 0.0, 0, 0], 'b_form_far': [0, 383.9109750000001, 1467.76, 1238.4225000000001], 'a_form_liab': [0.0, 0.0, 0, 0], 'b_form_liab': [0, 383.9109750000001, 1467.76, 1238.4225000000001], 'a_total_liability': -1569.0934750000001, 'b_total_liability'